<a href="https://colab.research.google.com/github/itsMatrik/fish-audio-google-colab-notebook/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fish Speech

### For Windows User / win用户

In [1]:
!chcp 65001

/bin/bash: line 1: chcp: command not found


### For Linux User / Linux 用户

In [2]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

In [3]:
!pip install -q huggingface_hub
import huggingface_hub
print("huggingface_hub version:", huggingface_hub.__version__)

huggingface_hub version: 0.36.0


In [4]:
from getpass import getpass
from huggingface_hub import login
hf_token = getpass("Вставьте ваш HF токен здесь (скрыто): ")
login(token=hf_token) # сохранит сессию в окружении Colab

Вставьте ваш HF токен здесь (скрыто): ··········


In [5]:
!git clone https://github.com/fishaudio/fish-speech.git

Cloning into 'fish-speech'...
remote: Enumerating objects: 6121, done.
remote: Counting objects: 100% (999/999), done.
remote: Compressing objects: 100% (273/273), done.
remote: Total 6121 (delta 856), reused 726 (delta 726), pack-reused 5122 (from 2)
Receiving objects: 100% (6121/6121), 19.78 MiB | 17.21 MiB/s, done.
Resolving deltas: 100% (3981/3981), done.


In [6]:
%cd fish-speech

/content/fish-speech


In [9]:
!apt-get install -y portaudio19-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libasound2-dev libjack-dev libjack0 libportaudio2 libportaudiocpp0
Suggested packages:
  libasound2-doc jackd1 portaudio19-doc
The following packages will be REMOVED:
  libjack-jackd2-0
The following NEW packages will be installed:
  libasound2-dev libjack-dev libjack0 libportaudio2 libportaudiocpp0
  portaudio19-dev
0 upgraded, 6 newly installed, 1 to remove and 41 not upgraded.
Need to get 596 kB of archives.
After this operation, 3,178 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libjack0 amd64 1:0.125.0-3build2 [93.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libasound2-dev amd64 1.2.6.1-1ubuntu1 [110 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libjack-dev amd64 1:0.125.0-3build2 [206 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/univers

In [10]:
!pip install -e .

Obtaining file:///content/fish-speech
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached datasets-2.18.0-py3-none-any.whl.metadata (20 kB)
  Using cached lightning-2.6.1-py3-none-any.whl.metadata (44 kB)
  Using cached hydra_core-1.3.2-py3-none-any.whl.metadata (5.5 kB)
  Using cached kui-1.14.0-py3-none-any.whl.metadata (982 bytes)
  Using cached loguru-0.7.3-py3-none-any.whl.metadata (22 kB)
  Using cached loralib-0.1.2-py3-none-any.whl.metadata (15 kB)
  Using cached pyrootutils-1.0.4-py3-none-any.whl.metadata (4.0 kB)
  Using cached resampy-0.4.3-py3-none-any.whl.metadata (3.0 kB)
  Using cached einx-0.2.2-py3-none-any.whl.metadata (6.8 kB)
  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build depende

### Prepare Model

In [11]:
# For Chinese users, you probably want to use mirror to accelerate downloading
# !set HF_ENDPOINT=https://hf-mirror.com
# !export HF_ENDPOINT=https://hf-mirror.com

!hf download fishaudio/openaudio-s1-mini --local-dir checkpoints/openaudio-s1-mini/

Fetching 7 files:   0% 0/7 [00:00<?, ?it/s]Downloading 'README.md' to 'checkpoints/openaudio-s1-mini/.cache/huggingface/download/Xn7B-BWUGOee2Y6hCZtEhtFu4BE=.7bc9c6ccc5e8a0f0ce233983a0f3da7d3345af34.incomplete'

README.md: 100% 2.86k/2.86k [00:00<00:00, 23.3MB/s]
Download complete. Moving file to checkpoints/openaudio-s1-mini/README.md

config.json: 100% 844/844 [00:00<00:00, 8.16MB/s]
Download complete. Moving file to checkpoints/openaudio-s1-mini/config.json

.gitattributes: 100% 1.52k/1.52k [00:00<00:00, 16.9MB/s]
Download complete. Moving file to checkpoints/openaudio-s1-mini/.gitattributes
Fetching 7 files:  14% 1/7 [00:00<00:03,  1.85it/s]
special_tokens.json:   0% 0.00/126k [00:00<?, ?B/s]

model.pth:   0% 0.00/1.74G [00:00<?, ?B/s]


codec.pth:   0% 0.00/1.87G [00:00<?, ?B/s]



special_tokens.json: 100% 126k/126k [00:00<00:00, 1.31MB/s]
Download complete. Moving file to checkpoints/openaudio-s1-mini/special_tokens.json




tokenizer.tiktoken: 100% 2.56M/2.56M [00:01<00:00, 2.5

## WebUI Inference

> You can use --compile to fuse CUDA kernels for faster inference (10x).

In [12]:
!python tools/run_webui.py \
    --llama-checkpoint-path checkpoints/openaudio-s1-mini \
    --decoder-checkpoint-path checkpoints/openaudio-s1-mini/codec.pth \
    # --compile

2026-01-31 20:40:57.017547: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769892057.038475    3388 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769892057.045302    3388 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769892057.060268    3388 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769892057.060291    3388 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769892057.060297    3388 computation_placer.cc:177] computation placer alr

## Break-down CLI Inference

### 1. Encode reference audio: / 从语音生成 prompt:

You should get a `fake.npy` file.

你应该能得到一个 `fake.npy` 文件.

In [ ]:
## Enter the path to the audio file here
src_audio = r"D:\PythonProject\vo_hutao_draw_appear.wav"

!python fish_speech/models/dac/inference.py \
    -i {src_audio} \
    --checkpoint-path "checkpoints/openaudio-s1-mini/codec.pth"

from IPython.display import Audio, display
audio = Audio(filename="fake.wav")
display(audio)

### 2. Generate semantic tokens from text: / 从文本生成语义 token:

> This command will create a codes_N file in the working directory, where N is an integer starting from 0.

> You may want to use `--compile` to fuse CUDA kernels for faster inference (~30 tokens/second -> ~300 tokens/second).

> 该命令会在工作目录下创建 codes_N 文件, 其中 N 是从 0 开始的整数.

> 您可以使用 `--compile` 来融合 cuda 内核以实现更快的推理 (~30 tokens/秒 -> ~300 tokens/秒)

In [ ]:
!python fish_speech/models/text2semantic/inference.py \
    --text "hello world" \
    --prompt-text "The text corresponding to reference audio" \
    --prompt-tokens "fake.npy" \
    --checkpoint-path "checkpoints/openaudio-s1-mini" \
    --num-samples 2
    # --compile

### 3. Generate speech from semantic tokens: / 从语义 token 生成人声:

In [ ]:
!python fish_speech/models/dac/inference.py \
    -i "codes_0.npy" \
    --checkpoint-path "checkpoints/openaudio-s1-mini/codec.pth"

from IPython.display import Audio, display
audio = Audio(filename="fake.wav")
display(audio)